In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from time import sleep
from datetime import date, timedelta

In [2]:
load_dotenv()
cid = os.getenv('client_id')
secret = os.getenv('client_secret')

#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


In [3]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
# track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_id)["items"]]

# for track in sp.playlist_tracks(playlist_id)["items"]:
#     #URI
#     track_uri = track["track"]["uri"]
    
#     #Track name
#     track_name = track["track"]["name"]
    
#     #Main Artist
#     artist_uri = track["track"]["artists"][0]["uri"]
#     artist_info = sp.artist(artist_uri)
    
#     #Name, popularity, genre
#     artist_name = track["track"]["artists"][0]["name"]
#     artist_pop = artist_info["popularity"]
#     artist_genres = artist_info["genres"]
    
#     #Album
#     album = track["track"]["album"]["name"]
    
#     #Popularity of the track
#     track_pop = track["track"]["popularity"]
    
# sp.audio_features(track_uri)[0]

playlist_id = playlist_link.split("/")[-1].split("?")[0]
username = 'Spotify'
results = sp.user_playlist(username, playlist_id, 'tracks')

In [4]:
playlist_tracks_data = results['tracks']
playlist_tracks_id = []
playlist_tracks_titles = []
playlist_tracks_artists = []
playlist_tracks_primary_artists = []

for track in playlist_tracks_data['items']:
    playlist_tracks_id.append(track['track']['id'])
    playlist_tracks_titles.append(track['track']['name'])
    # adds a list of all artists involved in the song to the list of artists for the playlist
    artist_list = []
    for artist in track['track']['artists']:
        artist_list.append(artist['name'])
    playlist_tracks_artists.append(artist_list)
    playlist_tracks_primary_artists.append(artist_list[0])

In [5]:
features = sp.audio_features(playlist_tracks_id)
# print(features)

In [6]:
songs_df = pd.DataFrame(data=features, columns=features[0].keys())

songs_df['title'] = playlist_tracks_titles
songs_df['primary_artist'] = playlist_tracks_primary_artists
songs_df['all_artists'] = playlist_tracks_artists

songs_df = songs_df[['id', 'title', 'primary_artist', 'all_artists',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'speechiness', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

songs_df["duration_s"] = songs_df.duration_ms.apply(lambda x : round(x/1000))
songs_df.drop("duration_ms", axis=1, inplace=True)
songs_df

,id,title,primary_artist,all_artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_s
0,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),Sam Smith,"[Sam Smith, Kim Petras]",0.714,0.472,2,-7.375,1,0.0864,0.013000,0.000005,0.2660,0.238,131.121,4,157
1,4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),David Guetta,"[David Guetta, Bebe Rexha]",0.561,0.965,7,-3.673,0,0.0343,0.003830,0.000007,0.3710,0.304,128.040,4,175
2,4Dvkj6JhhA12EX05fT7y2e,As It Was,Harry Styles,[Harry Styles],0.520,0.731,6,-5.338,0,0.0557,0.342000,0.001010,0.3110,0.662,173.930,4,167
3,2tTmW7RDtMQtBk7m2rYeSw,"Quevedo: Bzrp Music Sessions, Vol. 52",Bizarrap,"[Bizarrap, Quevedo]",0.621,0.782,2,-5.548,1,0.0440,0.012500,0.033000,0.2300,0.550,128.033,4,199
4,5ww2BF9slyYgNOk37BlC4u,La Bachata,Manuel Turizo,[Manuel Turizo],0.835,0.679,7,-5.329,0,0.0364,0.583000,0.000002,0.2180,0.850,124.980,4,163
5,6Sq7ltF9Qa7SNFBsV5Cogx,Me Porto Bonito,Bad Bunny,"[Bad Bunny, Chencho Corleone]",0.911,0.712,1,-5.105,0,0.0817,0.090100,0.000027,0.0933,0.425,92.005,4,179
6,7gRFDGEzF9UkBV233yv2dc,Shut Down,BLACKPINK,[BLACKPINK],0.818,0.690,0,-5.119,1,0.0454,0.003270,0.000000,0.1860,0.688,110.071,3,176
7,1IHWl5LamUGEuP4ozKQSXZ,Tití Me Preguntó,Bad Bunny,[Bad Bunny],0.650,0.715,5,-5.198,0,0.2530,0.099300,0.000291,0.1260,0.187,106.672,4,244
8,4h9wh7iOZ0GGn8QVp4RAOB,I Ain't Worried,OneRepublic,[OneRepublic],0.704,0.797,0,-5.927,1,0.0475,0.082600,0.000745,0.0546,0.825,139.994,4,148
9,5IgjP7X4th6nMNDh4akUHb,Under The Influence,Chris Brown,[Chris Brown],0.733,0.690,9,-5.529,0,0.0427,0.063500,0.000001,0.1050,0.310,116.992,4,185


In [7]:
# Add columns for genre
# Add columns for explicit

In [8]:
#write to csv
with open('./data/main.csv', 'w', encoding='utf-8') as f:
        songs_df.to_csv(f, header= True, index=True)